In [1]:
# import relevant libraries
import numpy as np
import pandas as pd

In [17]:
# read in hand history file 'hh.txt' via list comprehension
hh = [line.rstrip('\n') for line in open('hh.txt')]
# display the first 5 lines of hh
hh[:5]

['Winning Poker Network Game #1292108558: No Limit Holdem (75/150) [2018/10/16 06:50:34 UTC]',
 'Table: $10 Regular 9-Max, Table 1',
 'Tournament: 9100065',
 'Seats: 9',
 'Seat 1: Slizic (1,705)']

In [18]:
def get_stacks():
    ''' 
    get_stacks() extracts the names, stack sizes and position at the poker table from the 
    file hh.txt which is a text representation of an online poker tournament hand
    
    the output of get_stacks() is a dataframe which is to be passed to the icm
    calculator
    
    there are no side effects associated with this function
    '''
    # line.split()[1] is a string that i slice the first char with [:1]
    # creat a list of seats for players at the table read from hh.txt file
    seats = [line.split()[1][:1] for line in hh if '(' in line]
    # convert seats to int and drop first and last items in seats list
    seats = [int(seat) for seat in seats[1:-1]]
    
    # get names associated with each seat
    names = [line.split()[2:-1] for line in hh if '(' in line]
    names = [name for name in names[1:-1]]
    
    # get stacks associated with each seat
    stacks = [line.split()[-1] for line in hh if '(' in line]
    # drop parenthesis on each stack and drop the comma and convert stack to int
    # note: dropping first and last items in list as in seats
    stacks = [int(stack[1:-1].replace(',', '')) for stack in stacks[1:-1]]
    
    # identify which seat is the button
    # note this list comprehension only returns a one element list which I extract
    btn = [int(line.split()[-1]) for line in hh if 'Button is seat ' in line][0]
    
    index_btn = seats.index(btn)
    
    # create position list in appropriate deal order for 9 players PT4 convention
    pos = ['EP1','EP2','MP1','MP2','HJ','CO','BTN','SB','BB']
    pos = pos[9-len(stacks):]
    num_stacks = len(stacks)
    # repeat list to create a sliding window
    stacks1 = (stacks * 2)[index_btn:index_btn+num_stacks]
    names1 = (names * 2)[index_btn:index_btn+num_stacks]
    # join stacks to match deal order and positions
    stacks2 = stacks1[3:] + stacks1[:3]
    names2 = names1[3:] + names1[:3]
    df = pd.DataFrame({'pos': pos, 'name': names2, 'chips': stacks2}).set_index('pos')
    
    return df

get_stacks()

,name,chips
pos,,
HJ,[Zadrot],7089
CO,[Slizic],1705
BTN,[ShikeUB],1461
SB,[Davey_Flopit],842
BB,[unclepete55],2403


In [24]:
# for the purpose of my study I am calculating 9-man sng payout structure
# 0.5, 0.3, 0.2 to 1st, 2nd, 3rd respectively
def calc_ICM(df):
    '''
    calc_ICM() takes a dataframe with a column named chips that displays how many chips 
    each player has from the function get_stacks() 
    
    this function outputs a dataframe that displays each players portion of 
    equity in the total prizepool and also how likely it is that each person places
    1st, 2nd, and 3rd along with the combined probability that the player will make
    it in the money coming in 1st or 2nd or 3rd
    
    this is a dynamic approach for calculating ICM which is a method used to translate 
    chips to dollar values, originally used in horse racing, and featured in the 
    Mathematics of Poker by Bill Chen
    '''
    payouts = [0.5, 0.3, 0.2]
    tot = df.chips.sum()
    length = len(df.chips)
    # df.left1 is the amount of chips remaining if that player wins 1st place
    df['left1'] = tot - df.chips
    
    df['p1st'] = df.chips / tot
    
    
    # left2 is a df of chips remaining if players i and j get 1st and 2nd
    # note that who gets first and who gets second is irrelevant since the chips remaining
    #      will be the same
    left2 = pd.DataFrame(np.nan, index = range(length), columns = range(len(df.chips)))
    for i in range(len(df.chips)-1):
        for j in range(i+1,len(df.chips)):
            left2.iloc[i,j] = tot - df.chips.iloc[i] - df.chips.iloc[j]
    
    
    p2 = pd.DataFrame(np.nan, index = range(length), columns = range(length))
    # i is the row and represents the player placing first in the tournament
    for i in range(length):
        # p2[i,j] is the jth player's equity in 2nd place given player i is the winner
        for j in range(length):
            # if i == j then player cannot be second because she has already won
            if i != j:
                p2.iloc[i,j] = df.p1st.iloc[i] * df.chips.iloc[j] / df.left1.iloc[i] 
                
    df['eq1st'] = df.p1st * payouts[0]
    p2nd = [p2[i].sum() for i in range(length)]
    df['p2nd'] = p2nd
    df['eq2nd'] = df.p2nd * payouts[1]
    
    # multiply p2 and chips[i] / left2
    
    p3_list = []
    for ind, stack in enumerate(df.chips):
        p3 = pd.DataFrame(np.nan, index = range(length), columns = range(length))
        for i in range(length):
            for j in range(i+1, length):
                if (i != ind) & (j != ind):
                    p3.iloc[i,j] = stack / left2.iloc[i,j] * p2.iloc[i,j]
                    p3.iloc[j,i] = stack / left2.iloc[i,j] * p2.iloc[j,i]
        
        p3_list.append(p3.sum().sum())
    
    df['p3rd'] = p3_list
    df['eq3rd'] = df.p3rd * payouts[2]
    df['Equity%'] = (df.eq1st + df.eq2nd + df.eq3rd) * 100.0 
    df['pITM'] = df.p1st + df.p2nd + df.p3rd
    df['1st%'] = [p*100.0 for p in df.p1st]
    df['2nd%'] = [p*100.0 for p in df.p2nd]
    df['3rd%'] = [p*100.0 for p in df.p3rd]
    df['ITM%'] = [p*100.0 for p in df.pITM]
    return np.round(df[['name', 'chips', '1st%', '2nd%', '3rd%', 'ITM%', 'Equity%']], decimals=2)

# call calc_ICM and pass it the function get_stacks() which returns a dataframe, which is
#     what calc_ICM is expecting to receive
calc_ICM(get_stacks())



,name,chips,1st%,2nd%,3rd%,ITM%,Equity%
pos,,,,,,,
HJ,[Zadrot],7089,52.51,28.83,13.26,94.60,37.56
CO,[Slizic],1705,12.63,19.07,24.15,55.85,16.87
BTN,[ShikeUB],1461,10.82,16.59,22.16,49.57,14.82
SB,[Davey_Flopit],842,6.24,9.91,14.62,30.76,9.01
BB,[unclepete55],2403,17.80,25.60,25.81,69.21,21.74
